# Accidentes de tráfico en Reino Unido entre 2010 y 2014 

### Disponible en Kaggle en:
https://www.kaggle.com/stefanoleone992/adm-project-road-accidents-in-uk

### Variables y significado

* Accident_Index: Accident index
* Latitude: Accident latitude
* Longitude: Accident longitude
* Region: Accident region
* Urban_or_Rural_Area: Accident area (rural or urban)
* X1st_Road_Class: Accident road class
* Driver_IMD_Decile: Road IMD Decile
* Speed_limit: Road speed limit
* Road_Type: Road type
* Road_Surface_Conditions: Road surface condition
* Weather: Weather
* High_Wind: High wind
* Lights: Road lights
* Datetime: Accident datetime
* Year: Accident year
* Season: Accident season
* Month_of_Year: Accident month
* Day_of_Month: Accident day of month
* Day_of_Week: Accident day of week
* Hour_of_Day: Accident hour of day
* Number_of_Vehicles: Accident number of vehicles
* Age_of_Driver: Driver age
* Age_of_Vehicle: Vehicle age
* Junction_Detail: Accident junction detail
* Junction_Location: Accident junction location
* X1st_Point_of_Impact: Vehicle first point of impact
* Driver_Journey_Purpose: Driver journey purpose
* Engine_CC: Vehicle engine power (in CC)
* Propulsion_Code: Vehicle propulsion code
* Vehicle_Make: Vehicle brand
* Vehicle_Category: Vehicle brand category
* Vehicle_Manoeuvre: Vehicle manoeuvre when accident happened
* Accident_Severity: Accident severity

**Nombre completo del alumno:**  

**INSTRUCCIONES**: en cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). **No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("ModeloA")\
    .getOrCreate()

### Sobre el dataset anterior (accidents_uk.csv) se pide:

* **(1 punto)** Leerlo tratando de que Spark infiera el tipo de dato de cada columna, y cachearlo.

In [2]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
accidentesDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("modeloA accidents_uk.csv").cache()

In [3]:
from pyspark.sql.types import DoubleType
assert(accidentesDF.schema[1].dataType == DoubleType())

* **(1 punto)** Discretizar la variable **Age_of_Vehicle** utilizando un bucketizer (sin crear un pipeline) en los puntos de corte (0, 2, 5, 10, 15, 20, 35). La discretización debe quedar en una nueva columna de tipo Double llamada **Edad_Vehiculo**.

In [4]:
from pyspark.ml.feature import Bucketizer

bucketizer = Bucketizer()

# Python no tiene double lmaooo ya me joderia
bucketizer.setSplits([-float("inf"), 0, 2,5,10,15,20,35, float("inf")])
bucketizer.setInputCol("Age_of_Vehicle")
bucketizer.setOutputCol("Edad_Vehiculo")

accidentesBucketizedDF = bucketizer.transform(accidentesDF)

accidentesBucketizedDF.select(F.col("Edad_Vehiculo"),F.col("Age_of_Vehicle")).show(10)

+-------------+--------------+
|Edad_Vehiculo|Age_of_Vehicle|
+-------------+--------------+
|          3.0|             8|
|          2.0|             3|
|          3.0|             8|
|          2.0|             2|
|          4.0|            12|
|          2.0|             2|
|          4.0|            11|
|          3.0|             5|
|          1.0|             1|
|          2.0|             4|
+-------------+--------------+
only showing top 10 rows



In [5]:
assert("Edad_Vehiculo" in accidentesBucketizedDF.columns)
assert(accidentesBucketizedDF.schema.fields)

* **(1 punto)** Crear un nuevo DF donde la columna "Age_of_Driver" ha sido reemplazada por otra de tipo string en la que los valores 1 y 2 son "Adolescente", los valores 3 y 4 por "Joven", los valores 5 y 6 por "Adulto", y los demás valores se dejan sin modificar.

In [6]:
accidentesDF.select(F.col("Age_of_Driver")).show(5)

+-------------+
|Age_of_Driver|
+-------------+
|            4|
|            4|
|            7|
|            3|
|            5|
+-------------+
only showing top 5 rows



In [7]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
accidentesAgeDF = accidentesDF.withColumn("Age_of_Driver", F.when((F.col("Age_of_Driver")).between(1,2), "Adolescente")\
                                         .when((F.col("Age_of_Driver")).between(3,4), "Joven")\
                                         .when((F.col("Age_of_Driver")).between(5,6), "Adulto")\
                                         .otherwise(F.col("Age_of_Driver"))\
                                         )

In [8]:
assert(dict(accidentesAgeDF.dtypes)["Age_of_Driver"] == "string")
collectedDF = accidentesAgeDF.groupBy("Age_of_Driver").count().orderBy("count").collect()
print(collectedDF)
assert((collectedDF[0]["count"] == 9195) & (collectedDF[0]["Age_of_Driver"] == "8"))
assert((collectedDF[1]["count"] == 13338) & (collectedDF[1]["Age_of_Driver"] == "7"))
assert((collectedDF[2]["count"] == 57174) & (collectedDF[2]["Age_of_Driver"] == "Adolescente"))
assert((collectedDF[3]["count"] == 67138) & (collectedDF[3]["Age_of_Driver"] == "Adulto"))
assert((collectedDF[4]["count"] == 104987) & (collectedDF[4]["Age_of_Driver"] == "Joven"))

[Row(Age_of_Driver='8', count=9195), Row(Age_of_Driver='7', count=13338), Row(Age_of_Driver='Adolescente', count=57174), Row(Age_of_Driver='Adulto', count=67138), Row(Age_of_Driver='Joven', count=104987)]


* **(1 punto)** Partiendo de `accidentesDF`, crear un nuevo DataFrame de una sola fila que contenga, **por este orden de columnas**, el **número** de categorías existentes para el propósito del viaje, para el tipo de maniobra del vehículo, para las condiciones de la calzada y para la severidad del accidente. Pista: crear las columnas al vuelo con `select`(). Renombrar cada columna de conteo para que se llame igual que la propia columna que estamos contando.

In [9]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
numeroCategoriasDF = accidentesDF.select(countDistinct("Driver_Journey_Purpose").alias("Driver_Journey_Purpose")\
                                        ,countDistinct("vehicle_Manoeuvre").alias("vehicle_Manoeuvre")\
                                         ,countDistinct("Road_Surface_Conditions").alias("Road_Surface_Conditions")\
                                        ,countDistinct("Accident_Severity").alias("Accident_Severity"))
# YOUR CODE HERE

numeroCategoriasDF.show()

+----------------------+-----------------+-----------------------+-----------------+
|Driver_Journey_Purpose|vehicle_Manoeuvre|Road_Surface_Conditions|Accident_Severity|
+----------------------+-----------------+-----------------------+-----------------+
|                     5|               11|                      5|                2|
+----------------------+-----------------+-----------------------+-----------------+



In [10]:
assert(len(numeroCategoriasDF.columns) == 4)
assert(numeroCategoriasDF.count() == 1)

* **(3 puntos)** Partiendo de `accidentesAgeDF` definido anteriormente, crear un nuevo DataFrame con tantas filas como posibles propósitos de un viaje, y tantas columnas como rangos de edad habíamos distinguido en dicho DataFrame más una (la del propósito del viaje). Las columnas deben llamarse igual que cada uno de los niveles posibles de rangos de edad. Cada casilla del nuevo DataFrame deberá contener el **porcentaje** del número de accidentes ocurridos en ese tipo de viaje (fila) para ese rango de edad (columna), medido sobre el *total de accidentes ocurridos para ese tipo de viaje*.

Pista: se puede hacer todo en una sola secuencia de transformaciones sin variable auxiliar. Calcular primero el conteo, después añadir una columna con los totales de cada tipo de viaje como la suma de las 5 columnas de conteos, y finalmente reemplazar cada columna de conteo por su porcentaje. No debe utilizarse `when` en ningún momento, solo aritmética de columnas. Recuerda cómo desplegar grupos en varias columnas.

In [11]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
viajesPorEdadDF = accidentesAgeDF.groupBy("Driver_Journey_Purpose").pivot("Age_of_Driver")\
.agg(F.count("Age_of_Driver"))\
.withColumn("TotalAccidentes",F.col("7")+F.col("8")+F.col("Adolescente")+F.col("Joven")+F.col("Adulto"))\
.withColumn("7",F.col("7")/F.col("TotalAccidentes"))\
.withColumn("8",F.col("8")/F.col("TotalAccidentes"))\
.withColumn("Adolescente",F.col("Adolescente")/F.col("TotalAccidentes"))\
.withColumn("Joven",F.col("Joven")/F.col("TotalAccidentes"))\
.withColumn("Adulto",F.col("Adulto")/F.col("TotalAccidentes"))

viajesPorEdadDF.show()

+----------------------+--------------------+--------------------+-------------------+-------------------+-------------------+---------------+
|Driver_Journey_Purpose|                   7|                   8|        Adolescente|             Adulto|              Joven|TotalAccidentes|
+----------------------+--------------------+--------------------+-------------------+-------------------+-------------------+---------------+
|  Taking pupil to/f...|0.026578073089700997|0.009060706735125339|0.07550588945937783|0.22259136212624583|   0.66626396858955|           3311|
|  Pupil riding to/f...|0.021611001964636542| 0.01768172888015717| 0.6227897838899804|0.12573673870333987|0.21218074656188604|            509|
|  Journey as part o...|0.023330770595540836|0.003814536637293749|0.15536105032822758|0.31802590336507186|  0.499467739073866|          33818|
|       Other/Not known| 0.06503169065192967| 0.04814613718101034|0.24026836256901252| 0.2565088137105748|0.39004499588747266|         186017|

In [12]:
assert(len(viajesPorEdadDF.columns) >= 6)
assert("7" in viajesPorEdadDF.columns)
assert("8" in viajesPorEdadDF.columns)
assert("Adolescente" in viajesPorEdadDF.columns)
assert("Joven" in viajesPorEdadDF.columns)
assert("Adulto" in viajesPorEdadDF.columns)
assert(viajesPorEdadDF.columns[0] == "Driver_Journey_Purpose")
assert(viajesPorEdadDF.count() == 5)
commuting = viajesPorEdadDF.orderBy("Driver_Journey_Purpose").collect()[0]
assert(commuting.Driver_Journey_Purpose.startswith("Commuting"))
assert(abs(commuting['7'] - 0.012527948326649396) < 0.001)
assert(abs(commuting['8'] - 0.002519785640770) < 0.001)
assert(abs(commuting.Adolescente - 0.236327501153423) < 0.001)
assert(abs(commuting.Adulto - 0.2791993469851297) < 0.001)
assert(abs(commuting.Joven - 0.46942541789402703) < 0.001)

TypeError: Invalid argument, not a string or column: 0.0 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

* **(3 puntos)** Unir la información obtenida en el paso anterior al DataFrame `accidentesAgeDF`, de manera que **al resultado final se añada una columna nueva llamada `Porcentaje`** que contenga el porcentaje de accidentes que ha habido para ese rango de edad y ese tipo de viaje de entre todos los viajes que ha habido de ese tipo (es decir, el porcentaje adecuado de la tabla anterior). Por ejemplo, si el accidente se produjo en un trayecto de tipo `Commuting...` y la persona es `Joven`, entonces la columna Porcentaje tomará el valor de la columna `Joven` y por tanto tendrá el valor 0.46942, pero si la persona es `Adulto`, entonces tomará el valor de la columna `Adulto` el cual será 0.2791993469851297.

PISTA: unir los dos DF mediante join() convencional, y a continuación, crear la nueva columna `Porcentaje` en el resultado, utilizando `when` para ver cuál es el valor que debe tener en cada fila (más concretamente: de qué columna debemos tomarlo) en función del valor de la columna `Age_of_Driver`. No se necesitan variables intermedias; se puede hacer en una secuencia de transformaciones encadenadas.

In [ ]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
finalDF = viajesPorEdadDF.join(accidentesAgeDF, on =\
                              accidentesAgeDF["Driver_Journey_Purpose"] == viajesPorEdadDF["Driver_Journey_Purpose"])\
.withColumn("Porcentaje", F.when(F.col("Age_of_Driver") == "7", F.col("7"))\
            .when(F.col("Age_of_Driver") == "8", F.col("8"))\
            .when(F.col("Age_of_Driver") == "Adolescente", F.col("Adolescente"))\
            .when(F.col("Age_of_Driver") == "Joven", F.col("Joven"))\
            .when(F.col("Age_of_Driver") == "Adulto", F.col("Adulto")))
            
finalDF.select(viajesPorEdadDF["Driver_Journey_Purpose"],accidentesAgeDF["Age_of_Driver"], "Porcentaje").show(5)

In [ ]:
def sum_cond(df, column, condition): 
    return(df.where(condition).select(F.sum(column).alias(column)).collect()[0][column])
    
assert("Porcentaje" in finalDF.columns)
assert(abs(sum_cond(finalDF, "Porcentaje", F.col("Age_of_Driver") == "Adolescente") - 13344.826819125037) < 0.001)
assert(abs(sum_cond(finalDF, "Porcentaje", F.col("Age_of_Driver") == "Joven") - 44438.00809518224) < 0.001)
assert(abs(sum_cond(finalDF, "Porcentaje", F.col("Age_of_Driver") == "Adulto") - 18028.24488479408) < 0.001)
assert(abs(sum_cond(finalDF, "Porcentaje", F.col("Age_of_Driver") == "7") - 812.0952970292334) < 0.001)
assert(abs(sum_cond(finalDF, "Porcentaje", F.col("Age_of_Driver") == "8") - 432.2987413617681) < 0.001)